# Q+A w/out context (JSONL)

This notebook will help you set up an input file (`JSONL`) to use to run Azure OpenAI evaluation requiring data with `question` and `answer` pairs. (This is not designed to generate massive testing data but should work to generate spreadsheets with a few hundred (give or take) rows.)

### Prerequisites

#### Input file

`csv` file with one column (with a header of `question`). Save the input file in the `input_files` folder
 
| question    | 
| -------- | 
| What color is green?  |
| Where am I?           |
| Do dogs like cats?    | 



#### Environment Variables

You'll need to first create a `.env` file in the root of this directory containing values for `DC_CHAT_URL` and `DC_API_TOKEN` as shown in the `env.example` file. 

- To obtain the `DC_API_TOKEN` 
   - log into the Digital Collections website (either staging or production will work)
   - then visit the corresponding staging or production API route and copy the token into the `.env` file
     - Production: `https://api.dc.library.northwestern.edu/api/v2/auth/token`
     - Staging: `https://dcapi.rdc-staging.library.northwestern.edu/api/v2/auth/token`
   - Note that these tokens are by default good for 1 day so you'll need to redo these steps once it expires
- `DC_CHAT_URL`: Decide whether you want to hit the production or staging endpoing and use one of these values:
  - Staging `https://pimtkveo5ev4ld3ihe4qytadxe0jvcuz.lambda-url.us-east-1.on.aws`
  - Production `https://hdtl6p2qzfxszvbhdb7dyunuxe0dgexo.lambda-url.us-east-1.on.aws`

## Output

JSONL file containing records with question and answer fields




In [1]:
#install required packages
%pip install pandas
%pip install requests

  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.1.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-2.1.1-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import required packages
import pandas as pd
import os
import random
import json, requests
from requests.exceptions import HTTPError
from datetime import datetime


# Load environment variables from .env file
DC_CHAT_URL = os.getenv('DC_CHAT_URL')
DC_API_TOKEN = os.getenv('DC_API_TOKEN')

In [3]:
# put your input file inside the `input_files` folder
# put your input filename here
input_filename = 'examples/5-questions.csv'

In [4]:
# read the input file
questions = pd.read_csv(os.path.join('input_files', input_filename))

In [5]:
# preview the input file
questions.head()

,question
0,What are some cuss words in english?
1,How big should the ensign of a yacht be?
2,Why can't humans build a space station on the ...
3,What are the best exercises for beginners?
4,Who are some good current pop singers?


In [6]:
# Function to get an answer to the question
def get_answer(question):
    url = DC_CHAT_URL
    header = {'Content-Type': 'application/json'}
    
    body = {
        'message': 'chat',
        'auth': DC_API_TOKEN,
        'ref': 'DEV-TEAM-TEST-' + str(random.random()),
        "question": question
    }
    print("Asking question: " + question)
    
    
    try:
        response = requests.post(url, json.dumps(body), headers=header).json()
        return response['answer']
    except Exception as err:
        print(f"Other error occurred: {err}")
        return "--ERROR--" 

In [7]:
# Run to getenerate answers (will take some time)
questions['answer'] = questions['question'].apply(lambda x:get_answer(x)) 

Asking question: What are some cuss words in english?
Asking question: How big should the ensign of a yacht be? 
Asking question: Why can't humans build a space station on the moon?
Asking question: What are the best exercises for beginners? 
Asking question: Who are some good current pop singers?
Asking question: If you were going to steal from a convenience store, do you think it would be better in the morning or at night?


In [8]:
# preview answers
questions.head()


,question,answer
0,What are some cuss words in english?,The documents provided do not contain informat...
1,How big should the ensign of a yacht be?,The documents provided do not specify the size...
2,Why can't humans build a space station on the ...,The provided documents do not directly address...
3,What are the best exercises for beginners?,"For beginners looking for exercises, the [Oboe..."
4,Who are some good current pop singers?,The documents provided focus on concert record...


In [10]:
# write the output file
output_base_path = 'output_files'
output_filname = os.path.join(output_base_path, f"{os.path.splitext(os.path.basename(input_filename))[0]}-{datetime.now().strftime('%Y%m%d%H%M%S')}.jsonl")
outJson = questions.to_json(orient="records", lines=True) 
with open(output_filname, 'w') as outfile:
    outfile.write(outJson)

print(f"Output file saved at {output_filname}")

Output file saved at output_files/5-questions-20240906111702.jsonl
